In [150]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold

In [151]:
def read_data(folder, name1, name2, name3):
    loc = "%s/%s.txt" %(folder, name1)
    train_set = pd.read_csv(loc, sep="\t")
    print "Training set has %s rows and %s coumns" %(train_set.shape[0], train_set.shape[1])
    loc = "%s/%s.txt" %(folder, name2)
    test_set = pd.read_csv(loc, sep="\t")
    print "Testing set has %s rows and %s coumns" %(test_set.shape[0], test_set.shape[1])
    loc = "%s/%s.txt" %(folder, name3)
    val_set = pd.read_csv(loc, sep="\t")
    print "Validation set has %s rows and %s coumns" %(val_set.shape[0], val_set.shape[1])
    return train_set, test_set, val_set

In [152]:
train_set, test_set, val_set = read_data("data3", "train_data", "test_data", "valid_data")

Training set has 762 rows and 25 coumns
Testing set has 254 rows and 25 coumns
Validation set has 253 rows and 25 coumns


In [153]:
train_set_nb = sp.genfromtxt("data3/train_data.txt", delimiter="\t")
test_set_nb = test_set.values
val_set_nb = val_set.values

In [154]:
def np_cleaner(train, test, val):
    train = train[~np.isnan(train).any(axis=1)]
    print "Training set has %s rows and %s coumns" %(train.shape[0], train.shape[1])
    test = test[~np.isnan(test).any(axis=1)]
    print "Testing set has %s rows and %s coumns" %(test.shape[0], test.shape[1])
    val = val[~np.isnan(val).any(axis=1)]
    print "Validation set has %s rows and %s coumns" %(val.shape[0], val.shape[1])
    return train, test, val

In [155]:
train_data = train_set_nb[~np.isnan(train_set_nb).any(axis=1)]

In [156]:
train_set_nb, test_set_nb, val_set_nb = np_cleaner(train_data, test_set_nb, val_set_nb)

Training set has 761 rows and 25 coumns
Testing set has 254 rows and 25 coumns
Validation set has 253 rows and 25 coumns


In [157]:
train_all, test_all, val_all = np_cleaner(train_data, test_set_nb, val_set_nb)

Training set has 761 rows and 25 coumns
Testing set has 254 rows and 25 coumns
Validation set has 253 rows and 25 coumns


In [158]:
train_set.columns

Index([u'GP', u'GS', u'MIN', u'FGM', u'FGA', u'FG%', u'3PM', u'3PA', u'3P%',
       u'FTM', u'FTA', u'FT%', u'OFF', u'DEF', u'TRB', u'AST', u'STL', u'BLK',
       u'PF', u'TOV', u'PTS', u'YR', u'POS', u'W', u'H'],
      dtype='object')

In [159]:
columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 

# Labels

In [160]:
def np_labeliser(train, test, val, col):
    train = train[:,col]
    test = test[:,col]
    val = val[:,col]
    return train, test, val

In [161]:
train_labels, test_labels, val_labels = np_labeliser(train_set_nb, test_set_nb, val_set_nb, 22)

# Feature Selection

In [162]:
def np_featuriser(train, test, val,feature_list):
    
    train = np.delete(train,feature_list,1)
    test = np.delete(test,feature_list,1)
    val = np.delete(val,feature_list,1)
    
    return train, test, val

In [163]:
feature_list = [22]
train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

In [164]:
def vt_fsel(train):
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    sel.fit_transform(train)
    vt_list = sel.get_support()
    l_vt = []
    j = 0
    for i in vt_list:
        if i == False:
            l_vt.append(j)
            print "%s. featue name: %s" %(j, columns.keys()[columns.values().index(j)])
        j = j+1
    return l_vt    

In [165]:
list_vt = vt_fsel(train_all)

5. featue name: FG%
8. featue name: 3P%
11. featue name: FT%


In [166]:
from sklearn.feature_selection import SelectPercentile, f_classif
selector = SelectPercentile(f_classif, percentile=10)
X = train_features_nb
y = train_labels
#print X[:1]
selector.fit(X, y)
scores = -np.log10(selector.pvalues_)
#print scores
scores /= scores.max()
print scores

#for i in scores:
#scores

print "Most valuable features: "
remove = []
for i in range(0,25):
    remove.append(i)
    
j = 0
for i in scores:
    
    if i>0.20:
        print j, i, columns.keys()[columns.values().index(j)]
        remove.remove(j)
    if j == 21:
        j = j+2
    else:
        j = j+1
#for i in range(0,25)
print remove


[  3.45901528e-03   9.43715652e-04   2.45842793e-02   2.19370568e-02
   4.63643829e-02   5.25611868e-02   2.00699275e-01   2.29780197e-01
   2.40903355e-01   8.79857809e-03   1.63272775e-03   7.73922378e-02
   2.57437639e-01   8.75295104e-02   1.41504560e-01   2.47296954e-01
   1.10267123e-01   2.40587521e-01   5.15644450e-02   3.82173977e-02
   2.89519767e-02   2.90464311e-03   7.86685640e-01   1.00000000e+00]
Most valuable features: 
6 0.200699275139 3PM
7 0.229780197251 3PA
8 0.240903355016 3P%
12 0.257437639275 OFF
15 0.247296953898 AST
17 0.24058752131 BLK
23 0.786685640148 W
24 1.0 H
[0, 1, 2, 3, 4, 5, 9, 10, 11, 13, 14, 16, 18, 19, 20, 21, 22]


In [167]:
#feature_list = [0, 1, 2, 3, 4, 5, 9, 10, 11, 18, 19, 20, 21, 22]
#train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list)

In [168]:
#feature_list = [columns["GP"],columns["GS"],columns["MIN"],columns["FG%"],
#      columns["3P%"],columns["FT%"],columns["PTS"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]

#train_features_nb, test_features_nb, val_features_nb = np_featuriser(train_set_nb, test_set_nb, val_set_nb, feature_list )

# ALL Results

In [169]:
list_none = [22]
list_domain = [columns["GP"],columns["GS"],columns["MIN"],columns["FG%"],
      columns["3P%"],columns["FT%"],columns["PTS"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]
list_uni = [0, 1, 2, 3, 4, 5, 9, 10, 11, 13, 14, 16, 18, 19, 20, 21, 22]

In [170]:
def print_metrics(name, accuracy, conf_matrix):
    print "Feature selection: %s\n" %name
    print "Accuracy score: %s\n" %accuracy
    print "Confusion matrix:"
    print "\n%s" %conf_matrix
    print"\n"
    
def clf(clf, tr, tr_labels, val, val_labels):
        clf.fit(tr, tr_labels)
        
        pred = clf.predict(val)
        
        acc = metrics.accuracy_score(val_labels,pred)
        conf = metrics.confusion_matrix(val_labels, pred)
        
        return acc, conf

In [171]:
def clf_all(CLF, l_none, l_domain, l_uni, l_vt, train_all, test_all, val_all):
    tr_none, ts_none, val_none = np_featuriser(train_all, test_all, val_all, l_none)
    print tr_none.shape
    tr_domain, ts_domain, val_domain = np_featuriser(train_all, test_all, val_all, l_domain)
    
    tr_uni, ts_uni, val_uni = np_featuriser(train_all, test_all, val_all, l_uni)
    
    tr_vt, ts_vt, val_vt = np_featuriser(train_all, test_all, val_all, l_vt)
    
    #clfnb = GaussianNB()
    #print "Naive Bayes\n"
    
    acc_none, conf_none = clf(CLF, tr_none, train_labels, val_none, val_labels)
    print_metrics("None", acc_none, conf_none)
    
    acc_domain, conf_domain = clf(CLF, tr_domain, train_labels, val_domain, val_labels)
    print_metrics("Domain knowledge", acc_domain, conf_domain)

    acc_uni, conf_uni = clf(CLF, tr_uni, train_labels, val_uni, val_labels)
    print_metrics("Univariate", acc_uni, conf_uni)
    
    acc_vt, conf_vt = clf(CLF, tr_vt, train_labels, val_vt, val_labels)
    print_metrics("Variance Threshold", acc_vt, conf_vt)

# Naive Bayes

In [172]:
clfnb = GaussianNB()
clf_all(clfnb, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

(761L, 24L)
Feature selection: None

Accuracy score: 0.735177865613

Confusion matrix:

[[90  5  3]
 [12 63 42]
 [ 0  5 33]]


Feature selection: Domain knowledge

Accuracy score: 0.790513833992

Confusion matrix:

[[93  5  0]
 [18 74 25]
 [ 0  5 33]]


Feature selection: Univariate

Accuracy score: 0.786561264822

Confusion matrix:

[[90  8  0]
 [ 5 75 37]
 [ 0  4 34]]


Feature selection: Variance Threshold

Accuracy score: 1.0

Confusion matrix:

[[ 98   0   0]
 [  0 117   0]
 [  0   0  38]]




# SVM

In [173]:
svm = SVC()
svm.set_params(kernel='linear')
clf_all(svm, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

(761L, 24L)
Feature selection: None

Accuracy score: 0.893280632411

Confusion matrix:

[[ 93   5   0]
 [  6 105   6]
 [  0  10  28]]


Feature selection: Domain knowledge

Accuracy score: 0.893280632411

Confusion matrix:

[[ 93   5   0]
 [  7 103   7]
 [  0   8  30]]


Feature selection: Univariate

Accuracy score: 0.901185770751

Confusion matrix:

[[ 93   5   0]
 [  4 105   8]
 [  0   8  30]]


Feature selection: Variance Threshold

Accuracy score: 1.0

Confusion matrix:

[[ 98   0   0]
 [  0 117   0]
 [  0   0  38]]




# Logistic Regression 

In [174]:
logreg = linear_model.LogisticRegression(C=1e5)
clf_all(logreg, list_none, list_domain, list_uni, list_vt, train_all, test_all, val_all)

(761L, 24L)
Feature selection: None

Accuracy score: 0.849802371542

Confusion matrix:

[[ 89   9   0]
 [  8 100   9]
 [  0  12  26]]


Feature selection: Domain knowledge

Accuracy score: 0.877470355731

Confusion matrix:

[[ 91   7   0]
 [  6 104   7]
 [  0  11  27]]


Feature selection: Univariate

Accuracy score: 0.857707509881

Confusion matrix:

[[ 89   9   0]
 [  7 101   9]
 [  0  11  27]]


Feature selection: Variance Threshold

Accuracy score: 1.0

Confusion matrix:

[[ 98   0   0]
 [  0 117   0]
 [  0   0  38]]


